# Assignment 6 - Inferential Statistics
## Data Science Tools I
### Professor: Don Dalton

---

### Student: Duncan Ferguson

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

These questions will refer to a dataset on 801 Pokémon, taken from [this Kaggle link](https://www.kaggle.com/rounakbanik/pokemon). If you're not familiar with Pokémon, don't worry. You don't need any outside knowledge to do this assignment. If you are a Pokémon veteran, then you may know that there are actually more Pokémon than what are in this dataset, so this dataset is not complete. (You might say they have not caught them all.) Nonetheless, we will assume this dataset represents the entire population in question.

Normally we do not have access to the entire population being researched. This example is therefore unrealistic, even excluding the fact that we're talking about fictional monsters from a video game / anime series. Nonetheless, this demonstrates the ideas discussed in class and is hopefully fun for you.

# Question 1 - Exploratory Analysis

## Part (a)
### 2 points: Done

Download the `pokemon.csv` file from the Canvas assignment page and read it into a Pandas data frame. There are many columns that we can ignore; just keep the columns `'name'`, `'base_total'`, `'type1'`, `'type2'`, `'weight_kg'`, and `'generation'`. Print the head of your filtered data frame to confirm that you only have these columns. Also print the shape of the data frame to confirm that the dataset contains 801 rows.

In [8]:
df = pd.read_csv('pokemon.csv')
df = df[['name', 'base_total', 'type1', 'type2','weight_kg','generation']].copy()
df.head()

,name,base_total,type1,type2,weight_kg,generation
0,Bulbasaur,318,grass,poison,6.9,1
1,Ivysaur,405,grass,poison,13.0,1
2,Venusaur,625,grass,poison,100.0,1
3,Charmander,309,fire,NaN,8.5,1
4,Charmeleon,405,fire,NaN,19.0,1


In [9]:
df.shape

(801, 6)

## Part (b)
### 2 points

All Pokémon have either 1 or 2 types, called "primary" (corresponding to `'type1'`) and "secondary" (`'type2'`) respectively. As of the most recent games, there are 18 types total. Confirm this value using the column `'type1'`.

Then, count how many Pokémon there of each type. This can be easily computed using the `value_counts` method and by knowing that it is not possible for one Pokémon to have the same type for both `type1` and `type2`. Plot a bar plot showing a count for each type with each type listed along the x-axis.

## Part (c)
### 4 points

`'weight_kg'` (unsurprisingly) is the weight in kilograms of a given Pokémon. `'base_total'` is the "base stat total" of a Pokémon, which is the sum of its HP, attack, speed, etc. Simply put, the base total is a measure of a Pokémon's overall strength.

Plot separate distribution plots for the columns `'weight_kg'` and `'base_total'`, overlaying the estimated distribution (kde) on each histogram. Print the skew value for each column along with its plot.

Are these distributions normal? Include your answers as an in-line comment within your code cell(s).

# Question 2 - z-test

This question will use the `'base_total'` column as the data of interest and the Pokémon from Generation I as the sample set. While z-tests can easily be automated, here we will walk through the process to get a better feel for what's happening "behind the scenes".

## Part (a)
### 2 points

Suppose your friend likes Pokémon but is only familiar with the original 151 from generation 1. They didn't realize that many more games have come out since the first ones and are curious if the overall base totals have changed much with the newer generations. The Pokémon from generation 1 will be our sample.

For this example, we know the population mean and standard deviation. In realistic settings, these are given from outside sources, but in our case we can calculate them from the dataset of all Pokémon. Compute and print these values.

After determining the population mean (and standard deviation), state the null and alternative hypotheses in a text cell or as an in-line comment.

## Part (b)
### 2 points

As a separate data frame, extract the Pokémon that are from generation 1 to use as our sample. Call this variable `gen1`. Confirm that the last Pokémon included in the sample is the 151st (index 150) by printing the tail of `gen1`.

Calculate and print the standard error of the sampling distribution of the mean following the formula given in class:
$$
\sigma_{\bar{X}} = \frac{\sigma}{\sqrt{n}}
$$

## Part (c)
### 3 points

To demonstrate the properties of the z-distribution, create a Series that is the `'base_total'` column converted into z-scores using the formula:
$$
z = \frac{x-\mu}{\sigma}
$$
for each value $x$ in the column. Plot this distribution to visually confirm the mean is around 0 and the standard deviation is around 1.

Compute and output the "observed z", which is the sample mean converted into a z-score.
$$
z_{observed} = \frac{\bar{X} - \mu}{\sigma_{\bar{X}}}
$$

## Part (d)
### 2 points

Using a confidence level of $\alpha=0.05$ and corresponding critical values of $\pm1.96$, determine if the null hypothesis should be rejected or retained. State a conclusion regarding base totals of all Pokémon based on this decision rule.

YOUR ANSWER HERE

# Question 3 - t-test

Base totals again are the data of interest, but now with two independent samples of mostly male Pokémon from generation 1 and mostly female Pokémon from generation 1.

## Part (a)
### 3 points

As an example of a two-sample t-test, let's suppose we are interested in seeing if there is any bias coming from the designers of the games regarding the base totals for Pokémon that are more likely to be male and those that are more likely to be female (the intuition being that the base totals for mostly female Pokémon are perhaps lower than those of mostly males). The two-sample t-test will determine if the mean base totals between these two groups significantly differ.

Start by extracting the Pokémon from `gen1` whose `'percentage_male'` is strictly greater than 50 in a separate data frame. Do the same for mostly female Pokémon whose `'percentage_male'` is strictly less than 50. *Note that we are excluding Pokémon whose `'percentage_male'` is equal to 50.*

Confirm the normality of these two group's `'base_total'` values using distribution plots and skewness value as before.

## Part (b)
### 2 points

Generally the t-test requires an assumption that the variance between the two independent groups is roughly equal (called "homogeneity of variance"). The t-test method given by `stats` can actually account for when the variance is unequal as well. Regardless, we need to know which is the case for our data, so let's check using the Levene test. Again, we're interested in the `'base_total'` values of each group - mostly male and mostly female.

## Part (c)
### 3 points

Now that we know what to set the `equal_var` parameter to (either `True` or `False`), perform the t-test for two independent samples. Use the resulting p-value and an alpha of 0.05 to determine if the null hypothesis can be rejected or not. State your conclusion either as an in-line comment with your t-test code or as a separate text cell.

# Question 4 - ANOVA

Again we use base totals, but with three independent subsets of the population: Pokémon from generations 1, 2, and 3.

## Part (a)
### 3 points

Suppose another friend of yours was super into Pokémon for a while, playing the first three generations of the games for hours on end when they were a kid. They are curious if the base totals of Pokémon between generations stays relatively the same, or if there are perhaps generations that are "overpowered". To test this, we can use ANOVA.

Start by extracting the Pokémon from generations 2 and 3 into variables called `gen2` and `gen3`, just as we did earlier with `gen1`.

Plot all three generations on the same distribution plot using their generation value to adjust the color of the plot. This can be easily achieved by creating a combined data frame with all three generations appended one after the other (`gen1` appended with `gen2` appended with `gen3`). You can then use Seaborn's `displot` method to plot histograms of all three generations with `hue` set accordingly. Set the `multiple` parameter to `'dodge'` to make each bar sit side-by-side with the others.

## Part (b)
### 2 points

As we did for t-tests, test for equal variance among the three samples. For ANOVA, this ultimately is of little consequence since the F-test is robust to violation of the homogeneity of variance, but it's worth checking nonetheless. 

## Part (c)
### 3 points

Perform the ANOVA test on the base totals of generations 1, 2, and 3. Given an alpha of 0.05, include your conclusion as an in-line comment in your code or as a separate text cell.